# Example on the MNIST Fashion Dataset

### 1. Import of the modules and the dataset

In [1]:
from neural_network import *
from keras.datasets import fashion_mnist

2023-08-30 16:16:15.191255: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 2. Load the MNIST dataset

In [2]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

### 3. Build the neural network

In [3]:
net = NeuralNetwork(
    Normalization(samples=X_train),
    Reshape(output_shape=(1, 28, 28)),
    Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2),
    BatchNorm2d(6),
    Tanh(),
    AvgPool2d(kernel_size=(2, 2), stride=(2, 2)),
    Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0),
    BatchNorm2d(16),
    Tanh(),
    AvgPool2d(kernel_size=(2, 2), stride=(2, 2)),
    Flatten(),
    Linear(in_features=400, out_features=120),
    Tanh(),
    Linear(in_features=120, out_features=84),
    Tanh(),
    Linear(in_features=84, out_features=10),
    OutputLayer(activation_function="softmax", loss_function="categorical_cross_entropy")
)
# LeNet-5 alike architecture (see reference: http://d2l.ai/chapter_convolutional-modern/alexnet.html)
print(net)

NeuralNetwork:
 (0) Normalization(norm=255.0, dtype=float32)
 (1) Reshape(output_shape=(1, 28, 28))
 (2) Conv2d(in_channels=1, out_channels=6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), initialization=xavier)
 (3) BatchNorm2d(6, eps=1e-05, momentum=0.1)
 (4) Tanh()
 (5) AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
 (6) Conv2d(in_channels=6, out_channels=16, kernel_size=(5, 5), stride=(1, 1), padding=(0, 0), optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), initialization=xavier)
 (7) BatchNorm2d(16, eps=1e-05, momentum=0.1)
 (8) Tanh()
 (9) AvgPool2d(kernel_size=(2, 2), stride=(2, 2))
 (10) Flatten()
 (11) Linear(in_features=400, out_features=120, optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), initialization=xavier)
 (12) Tanh()
 (13) Linear(in_features=120, out_features=84, optimizer=Adam(lr=0.001, lr_decay=0.0, beta1=0.9, beta2=0.999, eps=1e-08), ini

### 4. Choose the number of samples to train on

In [4]:
# Select N samples to train on:
N = 3000
X_train, y_train = X_train[:N], y_train[:N]
X_train.shape, y_train.shape

((3000, 28, 28), (3000,))

### 5. Train the neural network

In [5]:
net.fit(X_train, y_train, epochs=5, batch_size=64, shuffle=True)

Training on 3000 samples:
Epoch    1 of 5    	 Average Error = 0.966801 	 Average Accuracy = 66.67%                                   
Epoch    2 of 5    	 Average Error = 0.585464 	 Average Accuracy = 79.30%                                   
Epoch    3 of 5    	 Average Error = 0.491926 	 Average Accuracy = 82.77%                                   
Epoch    4 of 5    	 Average Error = 0.441360 	 Average Accuracy = 85.10%                                   
Epoch    5 of 5    	 Average Error = 0.409686 	 Average Accuracy = 85.80%                                   
Training time : 00 hours, 00 minutes, 38 seconds


### 6. Make predictions on the test samples

In [6]:
# Prediction on the 10,000 test samples:
y_pred = net.predict(X_test, to="labels")
# Shows the first 10 predicted labels vs the true labels:
y_pred[:10], y_test[:10]

(array([9, 2, 1, 1, 6, 1, 4, 6, 5, 7]),
 array([9, 2, 1, 1, 6, 1, 4, 6, 5, 7], dtype=uint8))

### 7. Evaluate the accuracy score

In [7]:
print(f"Accuracy score on the test set: {accuracy_score(y_test, y_pred):.2%}")

Accuracy score on the test set: 80.83%


### 8. Build a confusion matrix

In [8]:
cm = confusion_matrix(y_test, y_pred)
print(f"Confusion matrix:\n{cm}")

Confusion matrix:
[[759   5  36  68  11   6 109   0   6   0]
 [  1 948   2  38   8   0   3   0   0   0]
 [  9   3 674  21 221   1  71   0   0   0]
 [ 20  20  14 885  31   0  29   0   1   0]
 [  0   2  88  49 812   0  46   0   3   0]
 [  0   0   0   2   0 957   0  31   1   9]
 [134   2 149  74 218   8 411   1   3   0]
 [  0   0   0   0   0  81   0 883   0  36]
 [  2   2  33   6  27  22  57   9 841   1]
 [  0   0   1   1   0  46   1  38   0 913]]


### 9. Displays a classification report

In [9]:
print(classification_report(cm, formatted=True))

Class: 0
Precision: 82.05%
Recall: 75.90%
F1-Score: 78.86%
Support: 1000

Class: 1
Precision: 96.54%
Recall: 94.80%
F1-Score: 95.66%
Support: 1000

Class: 2
Precision: 67.60%
Recall: 67.40%
F1-Score: 67.50%
Support: 1000

Class: 3
Precision: 77.36%
Recall: 88.50%
F1-Score: 82.56%
Support: 1000

Class: 4
Precision: 61.14%
Recall: 81.20%
F1-Score: 69.76%
Support: 1000

Class: 5
Precision: 85.37%
Recall: 95.70%
F1-Score: 90.24%
Support: 1000

Class: 6
Precision: 56.53%
Recall: 41.10%
F1-Score: 47.60%
Support: 1000

Class: 7
Precision: 91.79%
Recall: 88.30%
F1-Score: 90.01%
Support: 1000

Class: 8
Precision: 98.36%
Recall: 84.10%
F1-Score: 90.67%
Support: 1000

Class: 9
Precision: 95.20%
Recall: 91.30%
F1-Score: 93.21%
Support: 1000

